# Neural Network using YFinance data

In [ ]:
import yfinance as yf

# Download stock data from Yahoo Finance
stock_symbol = 'INFY.NS'  # Replace with the stock symbol of your choice
stock_data = yf.download(stock_symbol, start='2010-01-01', end='2024-02-26')

# Create a DataFrame from the downloaded data
df = pd.DataFrame(stock_data)
print(df)

## Using Ichimoku Cloud indicator

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adagrad
from keras.callbacks import EarlyStopping

# Calculate Ichimoku Cloud indicator
df['Tenkan-sen'] = df['Close'].rolling(window=9).mean()
df['Kijun-sen'] = df['Close'].rolling(window=26).mean()
df['Senkou Span A'] = (df['Tenkan-sen'] + df['Kijun-sen']) / 2
df['Senkou Span B'] = (df['High'].rolling(window=52).max() + df['Low'].rolling(window=52).min()) / 2
df['Chikou Span'] = df['Close'].shift(-26)
df = df[:-26]  # drop the last 26 rows
df.fillna(0, inplace=True)

# Create features for the neural network
features = df[['Tenkan-sen', 'Kijun-sen', 'Senkou Span A', 'Senkou Span B', 'Chikou Span']]

print(df)

In [ ]:
# Scale the features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Split the df into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, df['Close'], test_size=0.2, random_state=108)

# Create the neural network model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.4)) # Dropout
model.add(Dense(1000)) # Dense
model.compile(loss='mean_squared_error', optimizer=Adagrad(learning_rate=0.0001))

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[EarlyStopping(patience=10)])

# Make predictions
predictions = model.predict(X_test)

# Plot the predictions
plt.plot(y_test)
plt.plot(predictions)
plt.show()

In [ ]:
y_test_values = y_test.values
print(y_test_values)

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()
y_pred = y_pred[:657]

# Evaluate the model
mse = model.evaluate(X_test, y_test)
print("MSE: ", mse)

# Calculate mean absolute error
mae = mean_absolute_error(y_test_values, y_pred)
print("MAE:", mae)

# Calculate mean squared logarithmic error
from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(y_test_values, y_pred)
print("MSLE:", msle)

# Calculate coefficient of determination (R-squared)
from sklearn.metrics import r2_score
r2 = r2_score(y_test_values, y_pred)
print("R-squared:", r2)

# Plot actual vs predicted values
import matplotlib.pyplot as plt
plt.plot(y_test_values)
plt.plot(y_pred)
plt.legend(['Actual', 'Predicted'])
plt.show()

In [ ]:
print(model.summary())

In [ ]:
lstm_output = model.layers[0].output
dense_output = model.layers[2].output

print(lstm_output)
print(dense_output)